<a href="https://colab.research.google.com/github/fico-aditama/fad_001_data_mining_fake_news/blob/main/detect_fake_news_with_snorkel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deteksi Berita Palsu Bidang Kesehatan dengan Snorkel dan Perceptron

Proyek ini menggunakan pendekatan **Rule-Based Labeling** dengan **Snorkel** untuk memberi label data berita kesehatan secara otomatis, kemudian melatih model **Perceptron** untuk mendeteksi berita palsu.

## 1. Instalasi Library yang Diperlukan
Jalankan perintah berikut di terminal atau cell ini untuk menginstal library yang dibutuhkan:

In [1]:
!pip install snorkel==0.9.0 pandas scikit-learn numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 16.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


## 2. Import Library

In [ ]:
import pandas as pd
import numpy as np
from snorkel.labeling import labeling_function, PandasLFApplier, LabelModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import re

ImportError: cannot import name 'LabelModel' from 'snorkel.labeling' (/home/fadi/.local/lib/python3.10/site-packages/snorkel/labeling/__init__.py)

## 3. Memuat dan Preprocessing Data
Data yang diberikan adalah daftar judul berita dari Kompas.com. Kita akan memuatnya ke dalam DataFrame dan melakukan preprocessing sederhana.

In [ ]:
# Data yang diberikan dalam format tabel
data = {
    'No': list(range(1, 130)),
    'Judul Berita': [
        '"Tragedi Penghapusan \"Mandatory Spending\" dalam UU Kesehatan ..."',
        'Cara Pindah Faskes BPJS Kesehatan Online',
        'UU Kesehatan Terbaru: STR Dokter dan Perawat Berlaku Seumur ...',
        # Tambahkan semua judul berita dari dokumen Anda di sini
        # Untuk contoh, saya hanya masukkan beberapa
        'UU Kesehatan Baru Hapus Anggaran Wajib Minimal di Bidang ...',
        'UU Kesehatan Bolehkan Dokter Asing Berpraktik di RI, Kemenkes ...',
        # ... dst
    ],
    'Sumber': ['Kompas.com'] * 129,  # Sesuai jumlah data Anda
    'Tanggal': [
        '2023-08-13', '2024-07-13', '2023-07-11', '2023-07-12', '2023-07-15',
        # Tambahkan tanggal sesuai dokumen Anda
    ],
    'Kategori': ['Kesehatan'] * 129
}

# Buat DataFrame (untuk contoh ini, saya hanya masukkan 5 baris pertama)
df = pd.DataFrame(data[:5])

# Preprocessing: Membersihkan teks
def preprocess_text(text):
    text = text.lower()  # Ubah ke huruf kecil
    text = re.sub(r'[^\w\s]', '', text)  # Hapus tanda baca
    return text

df['Judul Berita'] = df['Judul Berita'].apply(preprocess_text)
df.head()

## 4. Definisi Labeling Functions dengan Snorkel
Kita akan membuat aturan sederhana untuk mendeteksi potensi berita palsu berdasarkan kata kunci sensasional atau kontradiksi.

In [ ]:
# Konstanta untuk label
ABSTAIN = -1
FAKE = 0
REAL = 1

# Labeling Function 1: Berita dengan kata sensasional dianggap potensi palsu
@labeling_function()
def sensational_keywords(x):
    keywords = ['tragedi', 'ramai', 'kontroversi', 'demo', 'tolak']
    return FAKE if any(keyword in x['Judul Berita'] for keyword in keywords) else ABSTAIN

# Labeling Function 2: Berita dengan panduan praktis dianggap asli
@labeling_function()
def practical_guide(x):
    keywords = ['cara', 'langkah', 'daftar', 'tips']
    return REAL if any(keyword in x['Judul Berita'] for keyword in keywords) else ABSTAIN

# Labeling Function 3: Berita dengan sumber resmi dianggap asli
@labeling_function()
def official_source(x):
    return REAL if 'kemenkes' in x['Judul Berita'] or 'jokowi' in x['Judul Berita'] else ABSTAIN

# Daftar semua labeling functions
lfs = [sensational_keywords, practical_guide, official_source]

## 5. Menerapkan Labeling Functions dengan Snorkel

In [ ]:
# Terapkan labeling functions ke data
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df)

# Gunakan LabelModel untuk menggabungkan hasil labeling functions
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=500, log_freq=100, seed=123)

# Prediksi label probabilistik
df['label'] = label_model.predict(L_train)
df.head()

## 6. Ekstraksi Fitur dan Pelatihan Model Perceptron
Kita akan mengubah teks menjadi fitur numerik menggunakan TF-IDF, lalu melatih model Perceptron.

In [ ]:
# Ekstraksi fitur dengan TF-IDF
vectorizer = TfidfVectorizer(max_features=500)
X = vectorizer.fit_transform(df['Judul Berita']).toarray()
y = df['label']

# Split data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Filter data yang memiliki label valid (bukan ABSTAIN)
mask_train = y_train != ABSTAIN
mask_test = y_test != ABSTAIN
X_train, y_train = X_train[mask_train], y_train[mask_train]
X_test, y_test = X_test[mask_test], y_test[mask_test]

# Latih model Perceptron
perceptron = Perceptron(max_iter=1000, tol=1e-3, random_state=42)
perceptron.fit(X_train, y_train)

# Prediksi pada data test
y_pred = perceptron.predict(X_test)

# Evaluasi
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:\n', classification_report(y_test, y_pred))

## 7. Contoh Prediksi pada Data Baru

In [ ]:
# Contoh prediksi pada judul berita baru
new_titles = [
    'cara cepat sembuh dari flu dengan bpjs kesehatan',
    'tragedi besar dokter asing bunuh pasien di ri'
]
new_titles_cleaned = [preprocess_text(title) for title in new_titles]
X_new = vectorizer.transform(new_titles_cleaned).toarray()
predictions = perceptron.predict(X_new)

for title, pred in zip(new_titles, predictions):
    label = 'Palsu' if pred == FAKE else 'Asli'
    print(f'Judul: {title} -> Prediksi: {label}')